<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
! wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
! unzip  concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

### 1. Import libraries, modules, and packages you will need. Make sure to import the preprocess_input function from keras.applications.vgg16

In [1]:
# Import the libraries needed.

import os
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

from PIL import Image

### 2. Use a batch size of 100 images for both training and validation. 

In [2]:
# Batch size of 100 images per batch.

batch_size_training = 100
batch_size_validation = 100

### 3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.

In [3]:
# 2 classes, Positive (cracked concrete) and Negative (non crackecd concrete).

num_classes = 2

# images are 224x224 pixels.

image_resize = 22

# Create an ImageDataGenerator object.

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

In [4]:
# Create a train_generator for passing the training images to the classifier.

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30000 images belonging to 2 classes.


In [5]:
# Create a validation_generator for passing the validation images to the classifier.

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9500 images belonging to 2 classes.


### 4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.

In [6]:
# Create the model and add the layers.

model = Sequential()
model.add(VGG16(include_top=False,pooling='avg',weights='imagenet',))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False

### 5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### 6. Fit the model on the augmented data using the ImageDataGenerators.

In [8]:
# Define the variables needed.

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

# Training will be for 2 epochs.

num_epochs = 2

In [9]:
# Fit the model.

fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
300/300 [==============================] - 7074s 24s/step - loss: 0.0945 - accuracy: 0.9674 - val_loss: 0.0244 - val_accuracy: 0.9941
Epoch 2/2
300/300 [==============================] - 6945s 23s/step - loss: 0.0192 - accuracy: 0.9955 - val_loss: 0.0144 - val_accuracy: 0.9966


In [10]:
# Save the model for future use.

model.save('classifier_VGG16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

### 1. Load your saved model that was built using the ResNet50 model.

In [13]:
# load resnet_50 model from week 3.

resnet50_model = keras.models.load_model('classifier_resnet_model.h5')

# load the vgg16_model.

vgg16_model=keras.models.load_model('classifier_VGG16_model.h5')

### 2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the shuffle parameter and set it to False.

In [15]:
# Create a test_generator for passing the test images to the classifiers.

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False)

Found 500 images belonging to 2 classes.


### 3. Use the evaluate_generator method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument.

In [20]:
vgg16_eval = vgg16_model.evaluate(test_generator)

16/16 [==============================] - 68s 4s/step - loss: 0.0118 - accuracy: 0.9980


In [21]:
resnet50_eval = resnet50_model.evaluate(test_generator)

16/16 [==============================] - 26s 2s/step - loss: 0.0022 - accuracy: 1.0000


### 4. Print the performance of the classifier using the VGG16 pre-trained model.

In [54]:
print("\nThe performance of the VGG16 pre-trained model:\n\nLoss is     :",
      round(vgg16_eval[0],4),"\nAccuracy is :",round(vgg16_eval[1],4),
      "\n\nSo the VGG16 model is ",round(vgg16_eval[1]*100,2),"% accurate")


The performance of the VGG16 pre-trained model:

Loss is     : 0.0118 
Accuracy is : 0.998 

So the VGG16 model is  99.8 % accurate


### 5. Print the performance of the classifier using the ResNet pre-trained model.

In [55]:
print("\nThe performance of the ResNet pre-trained model:\n\nLoss is     :",
      round(resnet50_eval[0],4),"\nAccuracy is :",round(resnet50_eval[1],4),
      "\n\nSo the ResNet model is ",round(resnet50_eval[1]*100,2),"% accurate")


The performance of the ResNet pre-trained model:

Loss is     : 0.0022 
Accuracy is : 1.0 

So the ResNet model is  100.0 % accurate


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

### 1.Use the predict_generator method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument.

In [56]:
# Predict the class of the test images using the VGG16 model.

vgg16_predict = vgg16_model.predict(test_generator)

# Predict the class of the test images using the Resnet50 model.

resnet50_predict = resnet50_model.predict(test_generator)

### 2. Report the class predictions of the first five images in the test set.

In [42]:
# Get the number of predictions.

rows = vgg16_predict.shape[0]

# Initilise list vgg16_yhat which will contain the predictions as "Positive" and "Negative"

vgg16_yhat = []

# Go through all the predictions and determine which class the prediction belongs to.

for row in range(rows):
    if vgg16_predict[row][0]>vgg16_predict[row][1]:  # If "Negative" has the higher probability then add "Negative" to list
        vgg16_yhat.append("Negative")
    else:
        vgg16_yhat.append("Positive")                # Else the prediction will be "Postive"

In [43]:
# Get the number of predictions.

rows = resnet50_predict.shape[0]

# Initilise list resnet50_yhat which will contain the predictions as "Positive" and "Negative".

resnet50_yhat = []

# Go through all the predictions and determine which class the prediction belongs to.

for row in range(rows):
    if resnet50_predict[row][0]>resnet50_predict[row][1]: # If "Negative" has the higher probability then add "Negative" to list
        resnet50_yhat.append("Negative")
    else:
        resnet50_yhat.append("Positive")                  # Else the prediction will be "Postive"

In [59]:
print("\nThe VGG16 model has predicted first five images from the test set are :\n\n",vgg16_yhat[:5],"\n")


The VGG16 model has predicted first five images from the test set are :

 ['Negative', 'Negative', 'Negative', 'Negative', 'Negative'] 



In [60]:
print("\nThe Resnet model has predicted first five images from the test set are :\n\n",resnet50_yhat[:5],"\n")


The Resnet model has predicted first five images from the test set are :

 ['Negative', 'Negative', 'Negative', 'Negative', 'Negative'] 



### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).